In [95]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split

In [121]:
import keras
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Activation
from keras.models import Sequential
from keras.utils import to_categorical

In [97]:
data = pd.read_csv('./pokemon_aug.csv')
data = data.drop(['Total', 'HP', 'Attack', 'Defense','Sp. Atk','Sp. Def',
                  'Speed', 'Generation', 'Legendary'], axis='columns')

In [113]:
mask = np.random.rand(len(data)) < 0.8

In [114]:
train = data[mask].reset_index()
test = data[~mask].reset_index()

In [115]:
gen = ImageDataGenerator(vertical_flip=True, validation_split=0.2)

In [116]:
traingen = gen.flow_from_dataframe(train, directory='./images/models/', x_col='fileid', y_col='Type1',
                                   class_mode='categorical', subset='training')
validgen = gen.flow_from_dataframe(train, directory='./images/models/', x_col='fileid', y_col='Type1',
                                   class_mode='categorical', subset='validation')

Found 253 images belonging to 18 classes.
Found 67 images belonging to 18 classes.


In [117]:
testdatagen = ImageDataGenerator(vertical_flip=False)
testgen = testdatagen.flow_from_dataframe(test, directory='./images/models', x_col='fileid', y_col='Type1')

Found 85 images belonging to 18 classes.


In [122]:
model = Sequential()
model.add(Conv2D(32, kernel_size=2, padding='same', activation='relu', 
                 input_shape=(256, 256, 3)))
model.add(Conv2D(32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=2, activation='relu'))
model.add(Conv2D(64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(18, activation='softmax'))

In [123]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [124]:
STEP_SIZE_TRAIN=traingen.n//traingen.batch_size
STEP_SIZE_VALID=validgen.n//validgen.batch_size
step_size_test=testgen.n//testgen.batch_size

In [ ]:
model.fit_generator(generator=traingen,
                    epochs=10,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=validgen,
                   validation_steps=STEP_SIZE_VALID)

Epoch 1/10
4/7 [================>.............] - ETA: 19s - loss: 14.9846

In [70]:
testgen.reset()
pred = model.predict_generator(testgen, steps = step_size_test, verbose=1)

2/2 [==============================] - 6s 3s/step


In [84]:
predicted_class_indices=np.argmax(pred,axis=1)

In [80]:
labels.items()

dict_items([(0, 'Grass'), (1, 'Fire'), (2, 'Water'), (3, 'Bug'), (4, 'Normal'), (5, 'Poison'), (6, 'Electric'), (7, 'Ground'), (8, 'Fairy'), (9, 'Fighting'), (10, 'Psychic'), (11, 'Rock'), (12, 'Ghost'), (13, 'Ice'), (14, 'Dragon'), (15, 'Dark'), (16, 'Steel'), (17, 'Flying')])

In [85]:
labels = (traingen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [86]:
len(predictions)

64

In [83]:
len(testgen.filenames)

91

In [81]:
filenames=testgen.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

ValueError: arrays must all be same length

In [ ]:
data.head()

In [ ]:
data.info()